In [57]:
import markdown
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Alignment, Font
from openpyxl.utils import get_column_letter
from openpyxl.utils.dataframe import dataframe_to_rows
import os
import re

In [58]:
 # convert md to plain text with all links

In [59]:
def markdown_to_rich_text(md):
    def replace_link(match):
        text, url = match.groups()
        return f'{text} ({url})'
    
    md = re.sub(r'\[([^]]+)\]\(([^)]+)\)', replace_link, md)
    md = md.replace("<br>", "\n").replace("</br>", "\n")
    return md

In [60]:
# extracting link text from md links
def extract_link_text(md):
    link_text = re.search(r'\[([^]]+)\]\(([^)]+)\)', md)
    return link_text.group(1) if link_text else md

In [61]:
# Read md file
input_file = "data_staging/groups.md"
with open(input_file, "r", encoding="utf-8") as file:
    markdown_content = file.read()

In [62]:
# Split md content into lines
lines = markdown_content.splitlines()

In [63]:
# Initialize vars to track table parsing
in_table = False
table_lines = []
header = None
data = []

In [64]:
# Process each line of md content
for line in lines:
    if line.strip().startswith("|"):
        if not in_table:
            in_table = True
            header = [cell.strip() for cell in line.split("|")[1:-1]]
        else:
            row_data = [cell.strip() for cell in line.split("|")[1:-1]]
            data.append(row_data)
    elif not in_table and line.strip().startswith("---------"):
        # try skiping lines that start with "---------" below header
        continue
    else:
        if in_table:
            break

In [65]:
# Create a DataFrame from the extracted table
if header and data:
    df = pd.DataFrame(data, columns=header)

In [66]:
    # Process table data, converting md to text with links
    df["DCG Name"] = df["DCG Name"].apply(markdown_to_rich_text)
    df["Website"] = df["Website"].apply(markdown_to_rich_text)
    df["Social Link / Point of Contact (POC)"] = df["Social Link / Point of Contact (POC)"].apply(markdown_to_rich_text)
    df["Join Group"] = df["Join Group"].apply(markdown_to_rich_text)

In [67]:
    # Create output /dir if it doesn't exist
    output_folder = "DCGroups-XLSX"
    os.makedirs(output_folder, exist_ok=True)

In [68]:
    # Create new dcg Excel with final groups data
    output_file = os.path.join(output_folder, "DCGroups-final.xlsx")
    wb = Workbook()
    ws = wb.active

In [69]:
    # Convert the DataFrame to Excel
    for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), 1):
        for c_idx, value in enumerate(row, 1):
            cell = ws.cell(row=r_idx, column=c_idx, value=value)
            cell.alignment = Alignment(wrap_text=True)  # Enable text wrapping for cells
            
            # Check if the cell value contains a Markdown link
            if isinstance(value, str) and re.search(r'\[([^]]+)\]\(([^)]+)\)', value):
                # Extract the link text and target URL
                link_text = extract_link_text(value)
                target_url = re.search(r'\[([^]]+)\]\(([^)]+)\)', value).group(2)
                
                # Format the cell as a clickable hyperlink with the link text
                cell.font = Font(color="0070C0", underline="single")
                cell.hyperlink = target_url

In [70]:
    # Adjust column widths for all columns
    for column in df.columns:
        max_length = df[column].astype(str).apply(len).max()  # Find the max length of values in each column
        col_letter = get_column_letter(df.columns.get_loc(column) + 1)  # Get the column letter
    
        # Adjust the column width to fit the max length plus some padding
        adjusted_width = max_length + 2
        ws.column_dimensions[col_letter].width = adjusted_width

In [71]:
    # Saving the Excel file
    wb.save(output_file)

In [72]:
print(f"Conversion complete. XLSX file saved to {output_file}")

Conversion complete. XLSX file saved to DCGroups-XLSX\DCGroups-final.xlsx
